In [1]:
# import the dependencies

import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [2]:
# filter out warning messages

warnings.filterwarnings('ignore')

# Read the CSV and Perform Basic Data Cleaning

In [3]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [5]:
# Create our features
X = df[[col for col in columns if col not in target]]
# Create our target
y = df[target]

In [6]:
# Encode application_type into two columns:
# - "application_type_Joint App", and ...

application_type_joint_app_dict = {
    "Individual":0,
    "Joint App":1
}
X["application_type_Joint App"] = X["application_type"].replace(application_type_joint_app_dict)

In [7]:
# ...
# - "application_type_Individual"

application_type_individual_dict = {
    "Individual":1,
    "Joint App":0
}
X["application_type_Individual"] = X["application_type"].replace(application_type_individual_dict)

In [8]:
# Encode next_paymnt_d into two columns:
# - "next_pymnt_d_Apr-2019", and ...

next_pymnt_d_Apr_dict = {
    "Apr-2019":1,
    "May-2019":0
}

X["next_pymnt_d_Apr-2019"] = X["next_pymnt_d"].replace(next_pymnt_d_Apr_dict)

In [9]:
# ...
# - "next_pymnt_d_May-2019"

next_pymnt_d_May_dict = {
    "Apr-2019":0,
    "May-2019":1
}

X["next_pymnt_d_May-2019"] = X["next_pymnt_d"].replace(next_pymnt_d_May_dict)

In [10]:
# Encode home_ownership

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

X["home_ownership"] = le.fit_transform(X["home_ownership"])

In [11]:
# Encode verification_status

X["verification_status"] = le.fit_transform(X["verification_status"])

In [12]:
# Encode issue_d

X["issue_d"] = le.fit_transform(X["issue_d"])

In [13]:
# Encode pymnt_plan

X["pymnt_plan"] = le.fit_transform(X["pymnt_plan"])

In [14]:
# Encode initial_list_status

X["initial_list_status"] = le.fit_transform(X["initial_list_status"])

In [15]:
# Encode hardship_flag into the new hardship_flag_N column

hardship_flag_N_dict = {
    "N":1,
    "Y":0
}
X["hardship_flag_N"] = X["hardship_flag"].replace(hardship_flag_N_dict)

In [16]:
# Encoding debt_settlement_flag into the new debt_settlement_flag_N column

debt_settlement_flag_N_dict = {
    "N":1,
    "Y":0
}
X["debt_settlement_flag_N"] = X["debt_settlement_flag"].replace(debt_settlement_flag_N_dict)

In [17]:
# Drop the label columns that were encoded above

X.drop(columns=["application_type","next_pymnt_d","home_ownership","pymnt_plan","initial_list_status","hardship_flag","debt_settlement_flag"],inplace=True)

In [18]:
# Encode loan_status

loan_status_dict = {
    "high_risk":1,
    "low_risk":0
}
y["loan_status"] = y["loan_status"].replace(loan_status_dict)

In [19]:
# Show the transformed X dataframe

X.describe()

,loan_amnt,int_rate,installment,annual_inc,verification_status,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,application_type_Joint App,application_type_Individual,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,0.669994,0.805542,21.778153,0.217766,0.497697,12.587340,...,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.139660,0.860340,0.383161,0.616839,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,0.719105,0.714932,20.199244,0.718367,0.758122,6.022869,...,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.346637,0.346637,0.486161,0.486161,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,...,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,0.000000,0.000000,13.890000,0.000000,0.000000,8.000000,...,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.000000,1.000000,0.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,1.000000,1.000000,19.760000,0.000000,0.000000,11.000000,...,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.000000,1.000000,0.000000,1.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,1.000000,1.000000,26.660000,0.000000,1.000000,16.000000,...,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.000000,1.000000,1.000000,1.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,2.000000,2.000000,999.000000,18.000000,5.000000,72.000000,...,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [20]:
# Check the balance of our target values

# 0 - Low Risk, 1 - High Risk

y['loan_status'].value_counts()

0    68470
1      347
Name: loan_status, dtype: int64

In [21]:
# Scale the X dataframe 

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [22]:
# Split the X and y into X_train, X_test, y_train, y_test

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_scaled,y,test_size=0.25)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [23]:
# Resample the training data with the RandomOversampler

# 0 - Low Risk, 1 - High Risk

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=4)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train['loan_status'])
Counter(y_resampled)

Counter({0: 51349, 1: 51349})

In [24]:
# Resample the training data with the RandomOversampler

from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(bootstrap=True, class_weight=None,
                criterion='gini', max_depth=None, max_features='auto',
                max_leaf_nodes=None, min_impurity_decrease=0.0,
                min_samples_leaf=2, min_samples_split=2,
                min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
                oob_score=False, random_state=1, replacement=False,
                sampling_strategy='auto', verbose=0, warm_start=False)
brf

BalancedRandomForestClassifier(min_samples_leaf=2, n_jobs=1, random_state=1)

In [25]:
# Predict y_pred from X_test

brf.fit(X_train, y_train)
y_pred = brf.predict(X_test)

In [26]:
# Calculate the balanced accuracy score

print("Accuracy score (validation): {0:.3f}".format(brf.score(X_test,y_test['loan_status'])))

Accuracy score (validation): 0.905


In [27]:
# Display the confusion matrix

from sklearn.metrics import confusion_matrix
y_pred = brf.predict(X_test)
matrix = confusion_matrix(y_test, y_pred)
matrix

array([[15511,  1610],
       [   32,    52]])

In [28]:
# Print the imbalanced classification report

# 0 - Low Risk, 1 - High Risk

from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.91      0.62      0.95      0.75      0.58     17121
          1       0.03      0.62      0.91      0.06      0.75      0.54        84

avg / total       0.99      0.90      0.62      0.95      0.75      0.58     17205



In [29]:
type(X)

pandas.core.frame.DataFrame

In [30]:
# List the features sorted in descending order by feature importance

sorted(zip(X.columns, brf.feature_importances_), reverse=True, key=lambda x: x[1])

[('last_pymnt_amnt', 0.07692290989682764),
 ('total_rec_prncp', 0.06790673290831657),
 ('total_pymnt', 0.06369482983930834),
 ('total_rec_int', 0.06328099611926143),
 ('total_pymnt_inv', 0.06297207436662343),
 ('int_rate', 0.03085377754140587),
 ('issue_d', 0.02449110223746132),
 ('dti', 0.02049759772253807),
 ('installment', 0.01734808356634381),
 ('mths_since_recent_inq', 0.01728652009487587),
 ('mo_sin_old_rev_tl_op', 0.016600742202423333),
 ('il_util', 0.01645169117461641),
 ('max_bal_bc', 0.015785516309303638),
 ('out_prncp', 0.01569538668785783),
 ('revol_bal', 0.015295186496005052),
 ('mths_since_rcnt_il', 0.015131776294711033),
 ('all_util', 0.014717188611422818),
 ('tot_cur_bal', 0.014146719364334103),
 ('total_bc_limit', 0.014142413255957168),
 ('bc_util', 0.013697103484749461),
 ('total_il_high_credit_limit', 0.013658855009225854),
 ('total_bal_il', 0.013595491738304523),
 ('total_rev_hi_lim', 0.013233185464543044),
 ('avg_cur_bal', 0.013174001964751283),
 ('out_prncp_inv', 

### Easy Ensemble AdaBoost Classifier

In [31]:
# Train the Classifier

from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(base_estimator=None, n_estimators=100, n_jobs=1,
            random_state=1, replacement=False, sampling_strategy='auto',
            verbose=0, warm_start=False)

eec.fit(X_train,y_train["loan_status"])


EasyEnsembleClassifier(n_estimators=100, n_jobs=1, random_state=1)

In [32]:
# Calculated the balanced accuracy score

print("Accuracy score (validation): {0:.3f}".format(eec.score(X_test,y_test['loan_status'])))

Accuracy score (validation): 0.942


In [33]:
# Display the confusion matrix

y_pred = eec.predict(X_test)
matrix = confusion_matrix(y_test, y_pred)
matrix

array([[16125,   996],
       [    9,    75]])

In [34]:
# Print the imbalanced classification report

# 0 - Low Risk, 1 - High Risk

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.94      0.89      0.97      0.92      0.85     17121
          1       0.07      0.89      0.94      0.13      0.92      0.84        84

avg / total       0.99      0.94      0.89      0.97      0.92      0.84     17205

